In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
# load and process dataset in this cell 
data_dir = 'vehicles/Veri Seti'
batch_size = 32
img_height = 128
img_width = 128


train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

In [ ]:
# display sample images in this cell with matplotlib

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
model = models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    layers.Conv2D(32, kernel_size=3, padding="same", activation="relu",
                  input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, kernel_size=3, padding="same", activation="relu"),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),

    layers.Dense(512, activation="relu"),
    layers.Dropout(0.3),

    layers.Dense(num_classes)   # logits (same as PyTorch)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)



In [ ]:
for epoch in range(EPOCHS):
    hist = model.fit(train_ds, validation_data=val_ds, epochs=1)
    train_loss = hist.history["loss"][0]
    train_acc = hist.history["accuracy"][0]
    val_acc = hist.history["val_accuracy"][0]

    print(f"Epoch {epoch+1}/{EPOCHS}, "
          f"Train Loss: {train_loss:.4f}, "
          f"Train Acc: {train_acc:.4f}, "
          f"Val Acc: {val_acc:.4f}")


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

print(f"Final Test Accuracy: {test_acc:.4f}")
print(f"Final Test Loss: {test_loss:.4f}")

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images, verbose=0)
    preds = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)


cm = confusion_matrix(y_true, y_pred)
print(cm)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()